In this notebook, we will make use of AzureML storage and compute resources for the training process.

In [1]:
from pathlib import Path
import pandas as pd
import tempfile
from joblib import dump

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

import mlflow
from azureml.core import Dataset, Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

In [2]:
from typing import Any
import matplotlib.pyplot as plt


In [3]:
DATA_DIR = Path("data/")

In [4]:
def create_dataset(data_dir):
    """Generate data from make_moons and save it to a folder.
    """
    X, y = make_moons(n_samples=1000, shuffle=True, noise=0.2, random_state=42)
    df = pd.DataFrame({"x1": X[:, 0], "x2": X[:, 1], "y": y})
    df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

    return df_train, df_test

In [5]:
df_train, df_test = create_dataset(DATA_DIR)

In [6]:
# Get workspace from config.
workspace = Workspace.from_config()
datastore = workspace.get_default_datastore()

In [7]:
workspace.datasets

{'DevDataset': DatasetRegistration(id='2d328165-e7d4-42a1-9f8d-43ce19d39d4c', name='DevDataset', version=1, description='', tags={}), 'moon_dataset': DatasetRegistration(id='72340cf3-b97f-479f-9822-9c3045247aa1', name='moon_dataset', version=1, description='', tags={})}

In [11]:
# Register the dataset to AzureML workspace
with tempfile.TemporaryDirectory() as tmp_dir:
    # We can only upload from disk, so we save it to a temporary directory.
    df_train.to_csv(f"{tmp_dir}/train.csv", index=False)
    df_test.to_csv(f"{tmp_dir}/test.csv", index=False)

    # Upload the datasets to the default datastore (blobstore container)
    # In this container it puts it inside the "datasets/moons" folder
    blob_store_path = "datasets/Dev/"
    Dataset.File.upload_directory(tmp_dir, (datastore, blob_store_path))
    
    # Create a dataset from the uploaded files and register it.
    dataset = Dataset.File.from_files(path=[(datastore, blob_store_path)])
    dataset.register(workspace, "DevDataset")

Validating arguments.
Arguments validated.
Uploading file to datasets/Dev/
Uploading an estimated of 2 files
Target already exists. Skipping upload for datasets/Dev/test.csv
Target already exists. Skipping upload for datasets/Dev/train.csv
Uploaded 0 files
Creating new dataset


In [12]:
dataset_name = "DevDataset"

In [13]:
dataset = Dataset.get_by_name(workspace=workspace, name=dataset_name)

# We can also download the data from AzureML
dataset.download(str(DATA_DIR), overwrite=True)

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/vm-1-training/code/Users/Prashant.Srivastava/azureml_experiment_tracking_tutorial/tutorials/experiment_tracking/data/test.csv',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/vm-1-training/code/Users/Prashant.Srivastava/azureml_experiment_tracking_tutorial/tutorials/experiment_tracking/data/train.csv']

Create a dataset in AzureML

In [14]:
azure_ws_mlflow_tracking_uri = workspace.get_mlflow_tracking_uri()
mlflow.set_tracking_uri(azure_ws_mlflow_tracking_uri)
mlflow.get_tracking_uri()

'azureml://westeurope.api.azureml.ms/mlflow/v1.0/subscriptions/8e155238-93f7-4377-9b62-6a2f4e51052e/resourceGroups/prashant-srivastava-sandbox/providers/Microsoft.MachineLearningServices/workspaces/azureml-mlflow?'

We will the job under an experiemnt alias.

In [19]:
experiment_name = "dev-train"

In [20]:
# We will use the same Experiment space that we used earlier.
experiment = Experiment(workspace, experiment_name)
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='', creation_time=1677848179318, experiment_id='60e8daa9-a127-4d3e-b658-66af46706393', last_update_time=None, lifecycle_stage='active', name='dev-train', tags={}>

Using compute engine for training the model. 

For this, we need:
- An AzureML compute intance (has already been created for you).
- A custom python environment for your compute instance.
- Scripts containing the training process.

In [21]:
compute_name = "vm-1-Training"

In [22]:
# Get your compute machine
ComputeTarget(workspace=workspace, name=compute_name)

Name,Workspace,State,Location,VmSize,Application URI,Docs
vm-1-Training,azureml-mlflow,Running,westeurope,STANDARD_DS1_V2,Jupyter JupyterLab,Doc


In [23]:
requirements_path = "../../requirements.txt"
environment_name = "dev-environment"

In [24]:
# Create a new custom environment
env = Environment.from_pip_requirements(
        name=environment_name,
        file_path=str(requirements_path),
)
# Python version must be added in this unclear way
env.python.conda_dependencies.set_python_version("3.8")


We need to run three scripts:
1. Download the dataset to our compute instace.
2. Run the training with MLflow.
3. Register the model to AzureML models.

First, we will create a script to download the dataset.

In [25]:
%%writefile download_dataset.py
from argparse import ArgumentParser
from azureml.core import Workspace, Dataset
parser = ArgumentParser()
parser.add_argument(
    '--dataset_name', 
    type=str, 
    default='moon_dataset', 
    help='The name of the dataset to download. The dataset must be registered in AzureML.')
parser.add_argument(
    '--output_folder', 
    type=str, 
    default='data', 
    help='The folder to save the dataset to.')
args = parser.parse_args()

workspace = Workspace.from_config()

dataset = Dataset.get_by_name(workspace, args.dataset_name)
dataset.download(args.output_folder, overwrite=True)

Writing download_dataset.py


Now, we will create a script for training the model with mlflow

In [26]:
%%writefile train_with_mlflow.py
import tempfile
from argparse import ArgumentParser
from typing import Any
from datetime import datetime

import mlflow
import pandas as pd
from joblib import dump
from azureml.core import Workspace
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

parser = ArgumentParser()
parser.add_argument("--train_dataset", type=str, default="data/train.csv")
parser.add_argument("--test_dataset", type=str, default="data/test.csv")
parser.add_argument("--n_cross_vals", type=int, default=5)
parser.add_argument("--max_depth", default=[2, 5, 10, None])
parser.add_argument("--n_estimators", default=[10, 25, 100])
parser.add_argument("--criterion", default=["gini", "entropy"])

args = parser.parse_args()
# This can be used to log all metrics and artifacts that are generated by the model.
# mlflow.autolog(log_models=True)
workspace = Workspace.from_config()

df_train = pd.read_csv(args.train_dataset)
df_test = pd.read_csv(args.test_dataset)

# We define the hyperparameters we want to tune
param_grid = {
    "n_estimators": args.n_estimators,
    "criterion": args.criterion,
    "max_depth": args.max_depth,
}
# We log the selected hyper-parameters to azureml using mlflow
# You can find the best hyper-parameters in the azureml UI under parameters.

mlflow.set_tracking_uri(workspace.get_mlflow_tracking_uri())

with mlflow.start_run(
    run_name=f"Moon_training-{datetime.now().strftime('%m/%d/%Y,%H:%M:%S')}"
) as run:
    run_id = run.info.run_id
    for param, value in param_grid.items():
        mlflow.log_param(f"gridsearch/{param}", str(value))

    model = RandomForestClassifier()
    grid_search = GridSearchCV(model, param_grid, cv=args.n_cross_vals, n_jobs=-1)

    # We train the model
    grid_search.fit(df_train[["x1", "x2"]], df_train["y"])
    model = grid_search.best_estimator_

    # Here we evaluate the model
    predictions = model.predict(df_test[["x1", "x2"]])
    test_accuracy = accuracy_score(df_test["y"], predictions)

    # We log the accuracy to azureml using mlflow
    # You can see the logged metrics in the azureml UI under the "Metrics" tab
    mlflow.log_metric("test_accuracy", test_accuracy)

    # We log the selected hyper-parameters to azureml using mlflow
    # You can find the best hyper-parameters in the azureml UI under parameters.
    for k, v in grid_search.best_params_.items():
        mlflow.log_param(f"selected/{k}", v)
    
    # Export the model and log it to azureml using mlflow
    with tempfile.TemporaryDirectory() as tmp_dir:
        dump(model, f"{tmp_dir}/model.joblib")
        mlflow.log_artifact(f"{tmp_dir}/model.joblib")


Writing train_with_mlflow.py


Finally, we will create a script to register the model.

In [27]:
%%writefile register_model.py
from argparse import ArgumentParser
import mlflow
from azureml.core import Workspace, Run

parser = ArgumentParser()
parser.add_argument(
    "--model_name", 
    type=str, 
    required=True, 
    help="Name of the model to register.")
parser.add_argument(
    "--run_id",
    type=str,
    default=None,
    required=False,
    help=(
        "ID AzureML has given the run."
        "You can find in the UI under raw json properties. "
        "If not provided, it will try to get it from the mlflow context."
    ),
)

args = parser.parse_args()

model_name = args.model_name
if args.run_id is not None:
    run_id =  args.run_id
run_id = Run.get_context(allow_offline=False)
run_id = run_id.id

ws = Workspace.from_config()
# Set the tracking URI to the AzureML workspace
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

# the artifact path is the path where the model artifact is stored within the run.
artifact_path = "model.joblib"
model_uri = f"runs:/{run_id}/{artifact_path}"
mlflow.register_model(model_uri=model_uri, name=model_name)


Writing register_model.py


Note that in all the three scripts, we are creating Workspace object from config. We need to keep the config file along the scripts, so that the scripts can use it for authenticating the Workspace. We can do this by: 

In [75]:
workspace.write_config(file_name='./config.json')

Now that we have the scripts, we can start running the script on out compute instance.

In [76]:
source_directory = "./"

In [77]:
commands = [
    "python download_dataset.py --dataset_name moon_dataset --output_folder data",
    "python train_with_mlflow.py --train_dataset data/train.csv --test_dataset data/test.csv",
    "python register_model.py --model_name moon_model",
]

In [78]:
# Here we combine all the configuration
script_run_config = ScriptRunConfig(
    source_directory=str(source_directory),
    command=" && ".join(commands),
    compute_target=compute_name,
    environment=env,
)

In [79]:
# Here we submit the configuration as an experiment job to AzureML.
experiment.submit(script_run_config)
print("You follow the experiment here:")
print(experiment.get_portal_url())

You follow the experiment here:
https://ml.azure.com/experiments/id/60e8daa9-a127-4d3e-b658-66af46706393?wsid=/subscriptions/8e155238-93f7-4377-9b62-6a2f4e51052e/resourcegroups/prashant-srivastava-sandbox/workspaces/azureml-mlflow&tid=3d4d17ea-1ae4-4705-947e-51369c5a5f79
